In [4]:
from selenium import webdriver # act like a web browser
from selenium.common.exceptions import WebDriverException, NoSuchElementException # import unclickable exception
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup # extract information from 

import time # slow down the algorithm
import re
scroll_height = 200
import datetime

In [3]:
chrome_driver_path_com9 = r'C:\Users\5842005226\Desktop\chromedriver.exe'
home_indy_path = r'E:\Downloads\Programs\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(home_indy_path)  # choose one path

options = Options()
options.headless = True
driver = webdriver.Chrome(home_indy_path)

# change the link here
agoda_link = 'https://www.agoda.com/pages/agoda/default/DestinationSearchResult.aspx?city=9395&checkIn=2019-04-20&los=1&rooms=1&adults=2&children=0&cid=-1&languageId=1&userId=7deafcf4-77d1-43a5-92b9-dd8f64158191&sessionId=jrdsi52uxzlsvmsjfidmyzar&pageTypeId=1&origin=TH&locale=en-US&aid=130243&currencyCode=THB&htmlLanguage=en-us&cultureInfoName=en-US&ckuid=7deafcf4-77d1-43a5-92b9-dd8f64158191&prid=0&checkOut=2019-04-21&priceCur=THB&textToSearch=Bangkok&travellerType=1&familyMode=off&productType=1&sort=reviewAll'
driver.get(agoda_link)

try:
    driver.maximize_window()
except WebDriverException:
    pass
driver.get_screenshot_as_file('capture.png')

True

In [5]:
#load data
import pickle
import os
# os.chdir(r'C:\Users\5842005226\Desktop')
data_static = pickle.load(open('data_static_21_13.pickle','rb'))

In [6]:
Hotel_name = []
Hotel_roomtype = []
Hotel_prices = []
Hotel_prices_standard = []
Hotel_benefits = []
Time_collect = []

In [7]:
def scroll_page_till_find(scroll_pause_time,last_height,scroll_height,element,driver):
    while True:
        first_height = last_height
        last_height += scroll_height
        driver.execute_script("window.scrollTo(" + str(first_height) + ", " + str(last_height) + ")")
        first_height = last_height
        time.sleep(scroll_pause_time)
        if abs(last_height - element.location['y']) <= scroll_height*3:
            break
    return last_height

In [8]:
def close_unused_tab(default_window_handle): # close the tab that is located on right side of the main tab
    handles = list(driver.window_handles)  
    assert len(handles) > 1
    handles.remove(default_window_handle)
    assert len(handles) > 0
    driver.switch_to_window(handles[0])
    driver.close()
    driver.switch_to_window(default_window_handle)

In [10]:
try:
    driver.maximize_window()
except WebDriverException:
    pass
continue_scrape = False ## if continue scrape <- change this to TRUE
while True:    
    if not continue_scrape:
        default_window_handle = driver.current_window_handle
        #hotel_id_in_our_database = 0
        last_height = 0
        element_index = 0
        click_elements = driver.find_elements_by_xpath('//h3[@class="hotel-name"]') # get the original click element
    while True:
        last_height = scroll_page_till_find(1.2,last_height,scroll_height,click_elements[element_index],driver)
        time.sleep(0.5) # wait for the data to be loaded
        click_elements_new = driver.find_elements_by_xpath('//h3[@class="hotel-name"]') # get the loaded data
        if len(click_elements_new) > len(click_elements): # if new data loaded -> update the data
            click_elements = click_elements_new # update the data
        try: # try to click the element
            click_elements[element_index].click() # click the element
        except WebDriverException: # if cannot -> close the popup
            try:
                popup_close = driver.find_element_by_xpath('//div[@class="LeaveSitePopup-CloseArea"]/span')
                popup_close.click() # close the popupb
            except NoSuchElementException:
                actions = ActionChains(driver)
                actions.move_by_offset(200,-40)
                actions.perform()
                print('no such element')
                time.sleep(0.5)
            
            click_elements_2 = driver.find_elements_by_xpath('//h3[@class="hotel-name"]')
            click_elements_2[element_index].click() # click the element again
        time.sleep(1) # browser wait
        driver.switch_to_window(driver.window_handles[1]) # switch to next window
        time.sleep(9)
        each_hotel_raw_html = driver.execute_script("return document.documentElement.outerHTML") # scrape the data
        soup = BeautifulSoup(each_hotel_raw_html,'lxml')
        try:
            hotel_name = soup.find('h1',{'class':"FirstTileContent__Title"}).get_text()
            hotel_all_room_type_info = soup.find('div',{'id':"property-room-grid-root"}).find('div',{'class':'RoomGrid-content'})
            try:
                if hotel_name in data_static['hotel_name'] and hotel_all_room_type_info.div['class'][0] != 'RoomGrid-searchTimeOut':# check whether 'all' hotel room is sold out or not
                    for hotel_room_type_info in hotel_all_room_type_info.contents: # 'all' room not sold out -> collect the data
                        hotel_room_type_span = hotel_room_type_info.find('span',{'data-selenium':'masterroom-title-name'})
                        if hotel_room_type_span is not None: # check whether hotel_room_type obj is normal or not
                            hotel_room_type_name = hotel_room_type_span.get_text()
                            hotel_room_type_info_prices_benefits = hotel_room_type_info.find('div',{'class':'ChildRoomsList'}).contents # get all prices & bentfit in each hotel_room_type 
                            for hotel_room_type_info_price_benefit in hotel_room_type_info_prices_benefits[1:]:
                                hotel_room_benefits = ''
                                hotel_room_type_price = hotel_room_type_info_price_benefit.find('div',{'class':'PriceContainer'})# find the price
                                hotel_room_type_benefits = hotel_room_type_info_price_benefit.find('div',{'class':"ChildRoomsList-room-featurebucket ChildRoomsList-room-featurebucket-Benefits"}).contents # find all benefits
                                for hotel_room_type_each_benefit in hotel_room_type_benefits[1:]: #find all benefits except 'benefit' head column
                                    hotel_room_benefits += hotel_room_type_each_benefit.find('span',{'class':"ChildRoomsList-roomFeature-TitleWrapper"}).get_text() + ';'
                                hotel_room_price_str = hotel_room_type_price.find('div',{'class':'finalPrice'}).find('strong',{'data-ppapi':"room-price"}).get_text() # extract room price
                                hotel_room_price_int = int(''.join(hotel_room_price_str.split(',')))
                                if hotel_room_type_price.find('div',{'class':"CrossedOutPrice "}) is None:
                                    hotel_room_price_standard = hotel_room_price_int
                                else:
                                    hotel_room_price_standard = float(hotel_room_type_price.find('div',{'class':"CrossedOutPrice "})["data-element-cor"])
                                Hotel_name.append(hotel_name)
                                Hotel_roomtype.append(hotel_room_type_name)
                                Hotel_prices.append(hotel_room_price_int)
                                Hotel_prices_standard.append(hotel_room_price_standard)
                                Hotel_benefits.append(hotel_room_benefits)
                                Time_collect.append(datetime.datetime.now())
                        else:
                            pass
                else:
                    pass
            except TypeError:
                pass
        except AttributeError: # if the page is not normal -> just close it
            pass
        driver.switch_to_window(default_window_handle)
        close_unused_tab(default_window_handle)
        element_index += 1
        if element_index >= len(click_elements): # if scroll down till find the last element -> break
            continue_scrape = False
            break
    try: # try to click the next-page button
        next_element = driver.find_element_by_xpath('//div[@class="clearfix pagination-panel"]/div/button[@id="paginationNext"]') # เสือกกด previous
        scroll_page_till_find(1.1,last_height,scroll_height,next_element,driver)
        #break
        next_element.click()
        time.sleep(5)
    except WebDriverException:
        break

no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element
no such element


IndexError: list index out of range

In [11]:
print(len(Hotel_name))

4579


In [12]:
from datetime import datetime

In [13]:
t_date = datetime.now()
today = t_date.isoformat()[:10]

In [14]:
d = {'Hotel_name':Hotel_name,'Hotel_roomtype':Hotel_roomtype,'Hotel_prices':Hotel_prices,
     'Hotel_prices_standard':Hotel_prices_standard,'Hotel_benefits':Hotel_benefits,'Time_collect':Time_collect}
filehandler = open("hotel_price_" + today +  ".pkl","wb")
pickle.dump(d,filehandler)
filehandler.close()

In [ ]:
close_unused_tab(default_window_handle)

In [ ]:
filehandler.close()

In [ ]:
close_unused_tab(default_window_handle)

In [ ]:
type(soup)

In [ ]:
soup.prettify()

In [ ]:
type(soup)

In [ ]:
hotel_all_room_type_info = soup.find('div',{'id':"property-room-grid-root"}).find('div',{'class':'RoomGrid-content'})

In [37]:
len(Hotel_name)

4797

In [28]:
hotel_room_type_name

'Executive Queen Suite'

In [25]:
hotel_room_price_standard

5550

In [27]:
hotel_room_price_int

5550

In [30]:
hotel_room_benefits

'Free breakfast for 2 ;Pay nothing until March 27, 2019 ;Free Wi-Fi ;Free cancellation before March 29, 2019 ;'

In [36]:
driver.quit()

In [14]:
print(datetime.datetime.now())

2019-03-25 15:46:49.064066


In [32]:
element_index += 30